In [1]:
search_for = 139
start_from = 0
threads = 10
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.23549175262451172
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 376006121
type: [1, 1, 1, 1, 139] 139
cases of this type: 373301041
100000 10.220344146895743
200000 16.629616797660493
300000 17.321217855572588
400000 17.113928173438506
500000 19.59737839589015
600000 17.064372190356668
700000 21.255544798962674
800000 18.423860376945036
900000 18.764950078030065
1000000 17.373670350487956
1100000 19.650946689180596
1200000 17.05867041503059
1300000 18.863799810951754
1400000 6.355779269929313
1500000 20.369165679931463
1600000 17.082538338581088
1700000 24.097617373892394
1800000 16.82540883483914
1900000 23.878308267361838
2000000 17.936453964837725
2100000 24.670087137278916
2200000 16.459338843092812
2300000 17.94734539480951
2400000 17.691433288879903
2500000 21.555232130641333
2600000 20.07847559589902
2700000 11.386015210970097
2800000 16.864211720086576
2900000 38.07384475792421
3000000

29700000 38.5083735471939
29800000 12.387480403256319
29900000 34.881666735199836
30000000 33.603747002365196
30100000 36.93819519456247
30200000 34.202416931381386
30300000 34.26496835014098
30400000 35.67458164326063
30500000 33.694261607130024
30600000 33.94451816408506
30700000 35.50994745748799
30800000 30.79987240484327
30900000 16.01474810622522
31000000 28.10609589800105
31100000 31.278712031258838
31200000 34.992880989730565
31300000 34.72802875950088
31400000 34.41805022142641
31500000 33.98701545178518
31600000 33.529967262880376
31700000 35.67746520706216
31800000 37.07930657400001
31900000 32.970709022858806
32000000 26.270645981718754
32100000 22.734427333261422
32200000 38.33880690985128
32300000 33.45764276161926
32400000 37.77944045366441
32500000 13.67927942519786
32600000 34.06920210477296
32700000 33.29329224572552
32800000 36.41775967980699
32900000 34.694759077289596
33000000 37.048880520761536
33100000 34.84373842083717
33200000 34.61822341282861
33300000 34.2367

59700000 30.233879835221185
59800000 32.29348942571793
59900000 29.218464874673163
60000000 31.24151081060317
60100000 31.012921593838065
60200000 27.891039631768887
60300000 29.55990845636644
60400000 23.64495306618067
60500000 12.600759574255497
60600000 29.851659791461913
60700000 28.998847616505156
60800000 29.713223769553654
60900000 29.50248310855688
61000000 30.84828163214913
61100000 30.16037365605534
61200000 31.207230105813107
61300000 29.788277318245914
61400000 12.895179522145474
61500000 32.981133444791034
61600000 32.50515815910257
61700000 31.638756329407272
61800000 34.036979628623754
61900000 33.39581240666153
62000000 33.55600003358968
62100000 35.06839200368152
62200000 27.683390010274334
62300000 17.93047662839343
62400000 31.494088372773728
62500000 33.770254879510574
62600000 31.853669910176247
62700000 34.92719880254207
62800000 32.09798004481901
62900000 29.300306241699904
63000000 31.62211331744641
63100000 20.87511038051804
63200000 18.28710878929387
63300000 

89500000 26.150272947295036
89600000 28.969466084014307
89700000 26.9382206188047
89800000 28.393712041385463
89900000 28.280970810974516
90000000 8.903143994415554
90100000 25.60219891861083
90200000 28.460534681868396
90300000 27.42432700488415
90400000 27.191500265531737
90500000 28.421593047148047
90600000 26.999849433784206
90700000 11.67223816250523
90800000 28.083218379983975
90900000 30.461040422569777
91000000 29.318445484764272
91100000 30.92731498710774
91200000 31.01845956507001
91300000 31.02396527947875
91400000 31.8418143669888
91500000 30.683904674504873
91600000 28.94078356836956
91700000 30.398724988693836
91800000 28.205611440219847
91900000 29.00523206607259
92000000 13.426325838047372
92100000 22.836771358026628
92200000 26.186636503850018
92300000 27.37149100701697
92400000 25.529709951993464
92500000 26.672417276967405
92600000 28.55713092812814
92700000 4.529969270670031
92800000 27.38255587967961
92900000 27.475429515198314
93000000 25.085958258336795
93100000 

118500000 26.27941777303921
118600000 25.261419559590625
118700000 25.616479552074786
118800000 23.676050510623494
118900000 24.21163509661638
119000000 20.465073183969423
119100000 10.807531455077266
119200000 22.925296971013946
119300000 22.660076794946384
119400000 23.354068986396403
119500000 17.085184343321643
119600000 13.593518824319048
119700000 23.1777158661
119800000 23.725926621763985
119900000 22.7610972860688
120000000 14.966232911009977
120100000 17.173818779997205
120200000 23.42545925850591
120300000 22.69626593662831
120400000 24.703646280520736
120500000 24.099440523286635
120600000 25.497325571404136
120700000 25.258592242064438
120800000 25.992015899118822
120900000 25.70835536123483
121000000 25.795105224591698
121100000 25.372960038726294
121200000 25.605152508561847
121300000 25.78531677291911
121400000 25.695193632274588
121500000 23.149377670968132
121600000 23.440628626889243
121700000 20.896798889337663
121800000 9.389713435606538
121900000 23.03525826585957


...

233200000 13.39238985404756
233300000 13.71560610416121
233400000 14.384496674336635
233500000 14.568958311787393
233600000 14.291193429938252
233700000 14.541005619617328
233800000 13.964533340221518
233900000 14.565935319741827
234000000 14.381328275387908
234100000 14.057521974868854
234200000 12.738826344787112
234300000 13.570378271417825
234400000 11.999834593686238
234500000 13.591675319525862
234600000 12.158673118596946
234700000 4.700263048813119
234800000 12.845705845677559
234900000 12.966079130604239
235000000 7.134086661549402
235100000 9.173729374720923
235200000 12.995416932049258
235300000 12.574425617839326
235400000 4.3545452929980835
235500000 13.347319498278015
235600000 12.389279285999464
235700000 13.262605035131818
235800000 13.170120498184904
235900000 13.2607950764003
236000000 13.639548516741616
236100000 14.271677805687911
236200000 14.265504294172942
236300000 13.524865165889564
236400000 14.115691494435822
236500000 13.097248328042198
236600000 14.39849520

261800000 11.092878266848754
261900000 1.7792729454991394
262000000 10.866764640406226
262100000 10.626613865393635
262200000 10.224945880381526
262300000 10.599747193034807
262400000 6.129749170654683
262500000 8.361439641879384
262600000 10.08692283350204
262700000 11.45741174798684
262800000 10.664629262651662
262900000 12.372107390509067
263000000 10.43456870131962
263100000 11.942361102554417
263200000 10.77508532084969
263300000 11.9364234115185
263400000 10.241834065221934
263500000 11.328730248725343
263600000 10.104105805536955
263700000 10.994279531975181
263800000 9.959540120861156
263900000 10.244016127966278
264000000 4.723454203148227
264100000 8.629350069183555
264200000 9.840159025276947
264300000 9.654744272942514
264400000 9.641082760873912
264500000 9.160891914861745
264600000 3.3001245518117432
264700000 9.672824836121285
264800000 10.04991022894376
264900000 9.185976405392994
265000000 9.84341789200084
265100000 6.239035534008453
265200000 7.447598476114571
2653000

291000000 6.879054449729157
291100000 7.873140336848271
291200000 7.108086626869167
291300000 7.84118729740552
291400000 4.224338485219326
291500000 5.752148457810074
291600000 7.009552981231548
291700000 7.953177808775653
291800000 7.063165873537552
291900000 7.990824115644568
292000000 7.532458436025161
292100000 8.367091648800452
292200000 2.5190959320860897
292300000 8.519195550729643
292400000 8.187688143941223
292500000 8.338266385023402
292600000 7.962451164640398
292700000 8.570892873347514
292800000 8.324054854755811
292900000 8.218283611809886
293000000 8.24002907102212
293100000 7.79671583867153
293200000 8.167963103679059
293300000 5.533085475784737
293400000 5.480942259893406
293500000 7.399311611521511
293600000 7.617984902743405
293700000 6.764212900797415
293800000 7.454960463147834
293900000 7.002448099011808
294000000 7.505171966098688
294100000 3.0743251924562744
294200000 6.647602098394399
294300000 6.839167250926596
294400000 7.705491359763049
294500000 6.763983864

320300000 5.767878063883651
320400000 5.114745377755928
320500000 5.567716093567815
320600000 5.2899755138066995
320700000 5.131585916732697
320800000 5.072211837882369
320900000 4.557762117068034
321000000 1.9466543207483378
321100000 5.012109668834062
321200000 4.9713734124791085
321300000 5.024905321538596
321400000 5.57100243147989
321500000 5.461718581343374
321600000 5.472153753627545
321700000 5.394087565086631
321800000 5.849415147042216
321900000 3.7364029407913963
322000000 3.9261096089073146
322100000 5.417917499848437
322200000 5.708982609514406
322300000 5.3358439049585895
322400000 5.633938236947996
322500000 5.195431180342037
322600000 5.036524583382108
322700000 2.3755802996082958
322800000 5.672380925539545
322900000 5.280707463990102
323000000 5.3542541247818
323100000 4.886158135949956
323200000 5.322013135403828
323300000 4.873155546164051
323400000 4.8590964772535115
323500000 4.6606296405335215
323600000 3.7259801683933915
323700000 2.502737672520862
323800000 4.6

349300000 1.6634264937293939
349400000 1.9669045003170968
349500000 2.5797639569067186
349600000 2.4880438436871484
349700000 2.4654617424644174
349800000 2.5771982805090996
349900000 2.361371106956587
350000000 2.492878274238783
350100000 2.340466538163915
350200000 2.4970925610125514
350300000 2.3328199650622565
350400000 2.34855467464635
350500000 0.9063581794760923
350600000 2.0567339176369845
350700000 2.3189943590382156
350800000 2.4764508828403864
350900000 2.227152241484237
351000000 2.4706754587121496
351100000 2.2317485417911054
351200000 2.5149408467907746
351300000 2.263921395535436
351400000 2.453554909675387
351500000 2.3077546618735494
351600000 2.5930053208481967
351700000 0.5615092871324924
351800000 2.535497844364393
351900000 2.3683326030397533
352000000 0.8754708779456489
352100000 2.357967752844727
352200000 2.4108523411299054
352300000 2.2765100765352266
352400000 2.388185003830159
352500000 2.366462745856638
352600000 2.2520730646098808
352700000 2.25546659048753